# Elhub API data - Gridloss - Summerproject 2025

## Visualization of Price area

Bjørn Eirik Rognskog Nordbak

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

oslo = ZoneInfo("Europe/Oslo")

def fetch_window(start_dt, end_dt):
    params = {
        "dataset":   "LOSS_PER_MBA_HOUR",
        "startDate": start_dt.isoformat(),
        "endDate":   end_dt.isoformat(),
    }
    url = "https://api.elhub.no/energy-data/v0/price-areas"
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    raw = resp.json().get("data", [])
    if not raw:
        print(f"  → no data for {start_dt.date()} → {end_dt.date()}, skipping")
        return pd.DataFrame()

    df = pd.json_normalize(
        raw,
        record_path=["attributes", "lossPerMbaHour"],
        meta=[
            ["attributes", "eic"],
            ["attributes", "name"],
            ["attributes", "status"],
        ],
        errors="ignore"
    ).rename(columns={
        "attributes.eic":    "eic",
        "attributes.name":   "name",
        "attributes.status": "status",
    })

    # parse as UTC, then convert to Oslo
    df["startTime"]       = pd.to_datetime(df["startTime"],       utc=True).dt.tz_convert(oslo)
    df["endTime"]         = pd.to_datetime(df["endTime"],         utc=True).dt.tz_convert(oslo)
    df["lastUpdatedTime"] = pd.to_datetime(df["lastUpdatedTime"], utc=True).dt.tz_convert(oslo)

    return df

# loop exactly as before
span_start = datetime(2023, 1, 1, tzinfo=oslo)
span_end   = datetime(2025, 6, 1, tzinfo=oslo)
window     = timedelta(days=7)

all_chunks = []
cur = span_start
while cur < span_end:
    nxt = min(cur + window, span_end)
    print(f"Fetching {cur.date()} → {nxt.date()}")
    all_chunks.append(fetch_window(cur, nxt))
    cur = nxt

big_df = pd.concat(all_chunks, ignore_index=True)
print("Total rows fetched:", len(big_df))


Fetching 2023-01-01 → 2023-01-08
Fetching 2023-01-08 → 2023-01-15
Fetching 2023-01-15 → 2023-01-22
Fetching 2023-01-22 → 2023-01-29
Fetching 2023-01-29 → 2023-02-05
Fetching 2023-02-05 → 2023-02-12
Fetching 2023-02-12 → 2023-02-19
Fetching 2023-02-19 → 2023-02-26
Fetching 2023-02-26 → 2023-03-05
Fetching 2023-03-05 → 2023-03-12
Fetching 2023-03-12 → 2023-03-19
Fetching 2023-03-19 → 2023-03-26
Fetching 2023-03-26 → 2023-04-02
Fetching 2023-04-02 → 2023-04-09
Fetching 2023-04-09 → 2023-04-16
Fetching 2023-04-16 → 2023-04-23
Fetching 2023-04-23 → 2023-04-30
Fetching 2023-04-30 → 2023-05-07
Fetching 2023-05-07 → 2023-05-14
Fetching 2023-05-14 → 2023-05-21
Fetching 2023-05-21 → 2023-05-28
Fetching 2023-05-28 → 2023-06-04
Fetching 2023-06-04 → 2023-06-11
Fetching 2023-06-11 → 2023-06-18
Fetching 2023-06-18 → 2023-06-25
Fetching 2023-06-25 → 2023-07-02
Fetching 2023-07-02 → 2023-07-09
Fetching 2023-07-09 → 2023-07-16
Fetching 2023-07-16 → 2023-07-23
Fetching 2023-07-23 → 2023-07-30
Fetching 2

In [2]:
big_df

,calculateLossFraction,calculatedLossQuantityKwh,consumptionQuantityKwh,endTime,exchangeInQuantityKwh,exchangeOutQuantityKwh,lastUpdatedTime,netInfeedQuantityKwh,priceArea,productionQuantityKwh,startTime,eic,name,status
0,0.057498,250340.00,4103527.2,2023-01-01 01:00:00+01:00,2941442.500,304687.70,2025-03-28 22:03:18+01:00,4353867.5,NO1,1717112.6,2023-01-01 00:00:00+01:00,10YNO-1--------2,NO1,NaN
1,0.058450,254941.90,4106783.5,2023-01-01 02:00:00+01:00,3143498.200,385411.80,2025-03-28 22:03:18+01:00,4361725.5,NO1,1603638.9,2023-01-01 01:00:00+01:00,10YNO-1--------2,NO1,NaN
2,0.059689,258876.08,4078219.8,2023-01-01 03:00:00+01:00,3340941.000,531545.10,2025-03-28 22:03:18+01:00,4337096.0,NO1,1527699.8,2023-01-01 02:00:00+01:00,10YNO-1--------2,NO1,NaN
3,0.059534,256737.36,4055726.0,2023-01-01 04:00:00+01:00,3382678.000,583037.44,2025-03-28 22:03:18+01:00,4312463.5,NO1,1512822.8,2023-01-01 03:00:00+01:00,10YNO-1--------2,NO1,NaN
4,0.058688,253150.06,4060329.2,2023-01-01 05:00:00+01:00,3381344.000,565922.70,2025-03-28 22:03:18+01:00,4313479.0,NO1,1498057.9,2023-01-01 04:00:00+01:00,10YNO-1--------2,NO1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105830,0.043892,84752.05,1846170.9,2025-05-31 20:00:00+02:00,57098.360,2457191.80,2025-06-14 18:34:39+02:00,1930922.9,NO5,4331016.5,2025-05-31 19:00:00+02:00,10Y1001A1001A48H,NO5,NaN
105831,0.043014,82610.73,1837946.4,2025-05-31 21:00:00+02:00,216415.580,2319910.20,2025-06-14 18:34:39+02:00,1920557.1,NO5,4024051.8,2025-05-31 20:00:00+02:00,10Y1001A1001A48H,NO5,NaN
105832,0.039738,75137.69,1815700.9,2025-05-31 22:00:00+02:00,147142.560,2119177.20,2025-06-14 18:34:39+02:00,1890838.6,NO5,3862873.2,2025-05-31 21:00:00+02:00,10Y1001A1001A48H,NO5,NaN
105833,0.041689,78273.39,1799296.4,2025-05-31 23:00:00+02:00,145239.640,2081319.50,2025-06-14 18:34:39+02:00,1877569.8,NO5,3813649.8,2025-05-31 22:00:00+02:00,10Y1001A1001A48H,NO5,NaN
